In [1]:
import numpy as np
import pickle
import json 
import random


In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()

In [3]:
intents = json.loads(open('intents.json').read())
remove_words = ['?',',','!']
words = []
classes = []
docs=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        pattern_list = nltk.word_tokenize(pattern)
        words.extend(pattern_list)
        docs.append((pattern_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        
        

In [4]:
print(docs)

[(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Hey', 'what', "'s", 'up'], 'greeting'), (['Hey', 'there'], 'greeting'), (['Who', 'are', 'you'], 'name'), (['What', "'s", 'your', 'name'], 'name'), (['How', 'can', 'I', 'call', 'you', '?'], 'name'), (['your', 'name', 'please'], 'name'), (['What', 'do', 'you', 'do', 'here', '?'], 'job'), (['What', 'are', 'you', 'doing'], 'job'), (['What', "'s", 'your', 'job', '?'], 'job'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['quit'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['fuck', 'you'], 'fuck'), (['wtf'], 'fuck'), (['shit'], 'fuck'), (['bullshit'], 'fuck'), (['holyfuck'], 'fuck'), (['holycrap'], 'fuck'), (['motherfucker'], 'fuck'), (['mf'], 'fuck'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'),

In [5]:
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in remove_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print(len(docs), len(words), len(classes))

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

51 71 14


In [6]:
traindata = []
output = [0 for _ in range (len(classes))]
for d in docs:
    bag = []
    
    #print(output)
    word_pattern = d[0]
    #print(word_pattern)
    word_pattern = [lemmatizer.lemmatize(w.lower()) for w in word_pattern]
    for w in words:
        if w in word_pattern:
            bag.append(1)
        else:
            bag.append(0)
    output_rows = list(output)
    output_rows[classes.index(d[1])] =1
    traindata.append([bag,output_rows])
random.shuffle(traindata)
traindata = np.array(traindata)

C:\Users\Vivek\anaconda3\lib\site-packages\ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [7]:
x_train = list(traindata[:,0])
y_train = list(traindata[:,1])

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM

model = Sequential()
model.add(Dense(128, input_shape= (len(x_train[0]),), activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(64,  activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(len(y_train[0]), activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               9216      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                910       
Total params: 18,382
Trainable params: 18,382
Non-trainable params: 0
_________________________________________________________________


In [8]:
from tensorflow.keras.optimizers import SGD
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer= 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
hist =  model.fit(np.array(x_train), np.array(y_train),epochs = 300, batch_size = 4, verbose = 1)

Epoch 1/300
13/13 [==============================] - 0s 614us/step - loss: 2.6046 - accuracy: 0.0392
Epoch 2/300
13/13 [==============================] - 0s 619us/step - loss: 2.6164 - accuracy: 0.0784
Epoch 3/300
13/13 [==============================] - 0s 688us/step - loss: 2.5810 - accuracy: 0.0980
Epoch 4/300
13/13 [==============================] - 0s 598us/step - loss: 2.5474 - accuracy: 0.1569
Epoch 5/300
13/13 [==============================] - 0s 726us/step - loss: 2.5218 - accuracy: 0.1765
Epoch 6/300
13/13 [==============================] - 0s 691us/step - loss: 2.4712 - accuracy: 0.2353
Epoch 7/300
13/13 [==============================] - 0s 612us/step - loss: 2.4274 - accuracy: 0.2157
Epoch 8/300
13/13 [==============================] - 0s 690us/step - loss: 2.4351 - accuracy: 0.2157
Epoch 9/300
13/13 [==============================] - 0s 764us/step - loss: 2.3077 - accuracy: 0.3137
Epoch 10/300
13/13 [==============================] - 0s 632us/step - loss: 2.3418 - accura

13/13 [==============================] - 0s 659us/step - loss: 0.1447 - accuracy: 0.9804
Epoch 82/300
13/13 [==============================] - 0s 690us/step - loss: 0.1371 - accuracy: 0.9804
Epoch 83/300
13/13 [==============================] - 0s 690us/step - loss: 0.1503 - accuracy: 0.9804
Epoch 84/300
13/13 [==============================] - 0s 657us/step - loss: 0.1732 - accuracy: 0.9608
Epoch 85/300
13/13 [==============================] - 0s 713us/step - loss: 0.0888 - accuracy: 1.0000
Epoch 86/300
13/13 [==============================] - 0s 612us/step - loss: 0.1592 - accuracy: 0.9412
Epoch 87/300
13/13 [==============================] - 0s 650us/step - loss: 0.1058 - accuracy: 0.9804
Epoch 88/300
13/13 [==============================] - 0s 653us/step - loss: 0.1044 - accuracy: 0.9804
Epoch 89/300
13/13 [==============================] - 0s 637us/step - loss: 0.1302 - accuracy: 0.9804
Epoch 90/300
13/13 [==============================] - 0s 596us/step - loss: 0.0736 - accuracy: 

13/13 [==============================] - 0s 614us/step - loss: 0.0289 - accuracy: 1.0000
Epoch 161/300
13/13 [==============================] - 0s 690us/step - loss: 0.0398 - accuracy: 1.0000
Epoch 162/300
13/13 [==============================] - 0s 746us/step - loss: 0.0638 - accuracy: 1.0000
Epoch 163/300
13/13 [==============================] - 0s 690us/step - loss: 0.0447 - accuracy: 1.0000
Epoch 164/300
13/13 [==============================] - 0s 537us/step - loss: 0.0104 - accuracy: 1.0000
Epoch 165/300
13/13 [==============================] - 0s 728us/step - loss: 0.0328 - accuracy: 1.0000
Epoch 166/300
13/13 [==============================] - 0s 614us/step - loss: 0.0391 - accuracy: 1.0000
Epoch 167/300
13/13 [==============================] - 0s 598us/step - loss: 0.0467 - accuracy: 0.9804
Epoch 168/300
13/13 [==============================] - 0s 767us/step - loss: 0.0304 - accuracy: 1.0000
Epoch 169/300
13/13 [==============================] - 0s 590us/step - loss: 0.0466 - a

13/13 [==============================] - 0s 690us/step - loss: 0.0115 - accuracy: 1.0000
Epoch 240/300
13/13 [==============================] - 0s 639us/step - loss: 0.0041 - accuracy: 1.0000
Epoch 241/300
13/13 [==============================] - 0s 684us/step - loss: 0.0106 - accuracy: 1.0000
Epoch 242/300
13/13 [==============================] - 0s 614us/step - loss: 0.0043 - accuracy: 1.0000
Epoch 243/300
13/13 [==============================] - 0s 599us/step - loss: 0.0542 - accuracy: 0.9804
Epoch 244/300
13/13 [==============================] - 0s 699us/step - loss: 0.0698 - accuracy: 0.9804
Epoch 245/300
13/13 [==============================] - 0s 573us/step - loss: 0.0206 - accuracy: 1.0000
Epoch 246/300
13/13 [==============================] - 0s 597us/step - loss: 0.0097 - accuracy: 1.0000
Epoch 247/300
13/13 [==============================] - 0s 537us/step - loss: 0.0093 - accuracy: 1.0000
Epoch 248/300
13/13 [==============================] - 0s 613us/step - loss: 0.0243 - a

In [9]:
model.save('chatbot.h5',hist)